**Inteligencia Artificial**

Sudoku: El objetivo del sudoku es rellenar una cuadrícula de 9x9 celdas dividida en subcuadrículas de 3x3 (también llamadas "cajas" o "regiones") con las cifras del 1 al 9 partiendo de algunos números ya dispuestos en algunas de las celdas. No se debe repetir ninguna cifra en una misma fila, columna o subcuadrícula. El n-sudoku corresponde al sudoku con NxN celdas, es decir el sudoku 9x9 corresponde al n-sudoku con n=9. Probar el algoritmo con N potencia de 3 (ej: 3, 9), especialmente con N=9. Utilizar juegos conocidos (De internet por ejemplo) o ver una estrategia para generar juegos válidos. Heurística posible: MVR (mínimo de valores restantes o también conocida como variable más restringida).

In [ ]:
import random
import pandas as pd
import numpy as np
import time
import sys

In [ ]:
sudoku_boards_df = pd.read_csv("https://raw.githubusercontent.com/MatiasLGonzalez/sudoku/main/sudoku2.csv")

puzzles = sudoku_boards_df["puzzle"]

puzzles

0      0700000430400096108006349000940520003584600200...
1      3010865040465210705000000014008000020803479000...
2      0483015603600080909106700030200009355090102006...
3      0083170000042051090000400703271609049014500000...
4      0408906300001368208007405190004670524500207002...
                             ...                        
994    0009040722090800100005129845007200000906080400...
995    0000740356020030000348560000060079040050600000...
996    5000030293400607500090500039000410864209000001...
997    0008090060000060080784025900000240074060859200...
998    0413000600084000509008000470271500000000020810...
Name: puzzle, Length: 999, dtype: object

Se han desarrollado tres métodos de utilidad que nos permiten llevar a cabo las siguientes acciones:

1. get_memory_usage: Este método permite obtener la cantidad de memoria en megabytes (MB) utilizada por un objeto.

2. performance_check_space: Este método tiene la función de verificar la cantidad de memoria en megabytes (MB) que se utiliza durante la ejecución de un algoritmo.

3. performance_check_time: Este método se encarga de verificar el tiempo que toma la ejecución de un algoritmo, expresado en segundos.

4. print_board: Este método nos facilita la impresión de la matriz.

5. extract_sudoku: Este método nos permite convertir un string de sudoku a una matriz

6. get_random_puzzle: Este método nos permite obtener un tablero de sudoku random de la lista

In [ ]:
def performance_check_time(board,algorithm):
  start_time = time.time()
  algorithm(board)
  end_time = time.time()
  return end_time - start_time

In [ ]:
def print_solution(algorithm,sudoku_board):
  result = algorithm(sudoku_board)
  if result:
    print("\nSolución:")
    print(np.matrix(sudoku_board))
  else:
    print("\nNo se pudo encontrar una solución.")

In [ ]:
def extract_sudoku(sudoku_string):
    matrix = []
    for i in range(9):
        row = [int(sudoku_string[i * 9 + j]) for j in range(9)]
        matrix.append(row)
    return matrix

In [ ]:
def get_random_puzzle():
    random_index = random.randint(0, len(puzzles) - 1)
    return extract_sudoku(puzzles[random_index])

Se crea una función que tiene la responsabilidad de validar la idoneidad de una celda en el contexto de un juego de Sudoku. Para asegurar la validez de la celda, es esencial verificar tanto la fila como la columna en la que se encuentra, además de la subcuadrícula 3 x 3 a la que pertenece. Este proceso garantiza que no haya duplicados de números en ninguno de estos contextos, lo que es fundamental para cumplir con las reglas del juego de Sudoku.

In [ ]:
def is_valid(board, row, col, num):
  N = len(board)
  # Verificamos la fila
  for i in range(N):
      if board[row][i] == num:
          return False
  # Verificamos la columna
  for i in range(N):
      if board[i][col] == num:
          return False
  # Verificamos el subcuadriculado
  subgrid_size = int(N ** 0.5)
  start_row, start_col = subgrid_size * (row // subgrid_size), subgrid_size * (col // subgrid_size)
  for i in range(subgrid_size):
    for j in range(subgrid_size):
      if board[i + start_row][j + start_col] == num:
        return False
  return True


En las siguientes líneas de código, procederemos a implementar tres algoritmos diferentes en cada caso. Los tres algoritmos que implementaremos son los siguientes:


---



1. Backtracking

In [ ]:
expanded_nodes_bt = 0

def backtracking_sudoku_solver(board):
  global expanded_nodes_bt
  n = len(board)
  for row in range(n):
    for col in range(n):
      if board[row][col] == 0:
        for num in range(1, n + 1):
          if is_valid(board, row, col, num):
            board[row][col] = num
            expanded_nodes_bt = expanded_nodes_bt + 1
            if backtracking_sudoku_solver(board):
              return True
            board[row][col] = 0
        return False
  return True


2. Algoritmo de Las Vegas

In [ ]:
expanded_nodes_lv = 0

def las_vegas_sudoku_solver(board):
  global expanded_nodes_lv
  empty_cell = find_empty_cell(board)
  if not empty_cell:
    return True
  row, col = empty_cell
  numbers = list(range(1, 10))
  random.shuffle(numbers)
  for num in numbers:
    if is_valid(board, row, col, num):
      board[row][col] = num
      expanded_nodes_lv = expanded_nodes_lv + 1
      if las_vegas_sudoku_solver(board):
        return True
      board[row][col] = 0
  return False

def find_empty_cell(board):
  for i in range(9):
    for j in range(9):
      if board[i][j] == 0:
          return (i, j)
  return None



3. Algoritmo Heurístico (utilizando MVR - Mínimo de Valores Restantes)

In [ ]:
expanded_nodes_csp = 0

def csp_sudoku_solver(board):
  global expanded_nodes_csp
  mcv = most_constrained_cell(board)
  if not mcv:
      return True

  row, col = mcv

  for num in possible_values(board, row, col):
      board[row][col] = num
      expanded_nodes_csp = expanded_nodes_csp + 1
      if csp_sudoku_solver(board):
          return True
      board[row][col] = 0

  return False

def possible_values(board, row, col):
  values = set(range(1, 10))
  values -= set(board[row])
  values -= set([board[i][col] for i in range(9)])
  start_row, start_col = 3 * (row // 3), 3 * (col // 3)
  values -= set([board[start_row + i][start_col + j] for i in range(3) for j in range(3)])
  return values

def most_constrained_cell(board):
  min_possible_values = 10
  mcv = None
  for i in range(9):
    for j in range(9):
      if board[i][j] == 0:
        num_possible_values = len(possible_values(board, i, j))
        if num_possible_values < min_possible_values:
          min_possible_values = num_possible_values
          mcv = (i, j)
  return mcv

Realizamos las pruebas sobre un Tablero de Pruebas

1. Backtracking

In [ ]:
sudoku_board = get_random_puzzle()

print("Tablero de Sudoku Original:")
print(np.matrix(sudoku_board))

print_solution(backtracking_sudoku_solver,sudoku_board)

Tablero de Sudoku Original:
[[0 4 9 0 1 2 6 5 7]
 [6 1 5 0 4 0 0 0 0]
 [0 8 2 6 5 0 1 0 0]
 [8 7 4 0 0 1 0 6 3]
 [2 3 0 5 0 0 0 1 9]
 [9 0 1 7 3 6 4 0 2]
 [0 6 3 0 0 8 9 0 5]
 [4 9 8 1 0 5 0 3 0]
 [0 2 7 0 0 9 0 4 1]]

Solución:
[[3 4 9 8 1 2 6 5 7]
 [6 1 5 9 4 7 3 2 8]
 [7 8 2 6 5 3 1 9 4]
 [8 7 4 2 9 1 5 6 3]
 [2 3 6 5 8 4 7 1 9]
 [9 5 1 7 3 6 4 8 2]
 [1 6 3 4 2 8 9 7 5]
 [4 9 8 1 7 5 2 3 6]
 [5 2 7 3 6 9 8 4 1]]


In [ ]:
sudoku_board = get_random_puzzle()

print("Tablero de Sudoku Original:")
print(np.matrix(sudoku_board))

print_solution(las_vegas_sudoku_solver,sudoku_board)

Tablero de Sudoku Original:
[[0 0 0 0 1 0 2 5 8]
 [0 0 1 0 0 7 0 3 0]
 [0 4 2 0 3 0 1 0 7]
 [5 9 3 4 7 2 6 0 0]
 [0 0 8 0 0 0 0 0 3]
 [1 0 6 0 8 0 0 4 0]
 [0 3 9 0 0 0 0 0 2]
 [0 1 0 5 0 0 0 0 0]
 [7 0 5 0 6 0 3 9 0]]

Solución:
[[3 6 7 9 1 4 2 5 8]
 [8 5 1 6 2 7 4 3 9]
 [9 4 2 8 3 5 1 6 7]
 [5 9 3 4 7 2 6 8 1]
 [4 7 8 1 5 6 9 2 3]
 [1 2 6 3 8 9 7 4 5]
 [6 3 9 7 4 8 5 1 2]
 [2 1 4 5 9 3 8 7 6]
 [7 8 5 2 6 1 3 9 4]]


In [ ]:
sudoku_board = get_random_puzzle()

print("Tablero de Sudoku Original:")
print(np.matrix(sudoku_board))

print_solution(csp_sudoku_solver,sudoku_board)

Tablero de Sudoku Original:
[[0 2 0 0 0 5 6 0 3]
 [7 0 3 0 1 2 8 0 0]
 [8 0 0 0 4 0 0 2 0]
 [0 8 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 4 0 0]
 [0 1 0 0 0 8 0 0 5]
 [0 0 1 0 3 9 0 0 2]
 [0 0 0 0 5 1 0 3 0]
 [0 0 9 0 6 4 0 5 0]]

Solución:
[[1 2 4 9 8 5 6 7 3]
 [7 5 3 6 1 2 8 4 9]
 [8 9 6 3 4 7 5 2 1]
 [4 8 7 5 9 3 2 1 6]
 [9 3 5 1 2 6 4 8 7]
 [6 1 2 4 7 8 3 9 5]
 [5 4 1 8 3 9 7 6 2]
 [2 6 8 7 5 1 9 3 4]
 [3 7 9 2 6 4 1 5 8]]


Realizamos una prueba de rendimiento teniendo en cuenta el tiempo y el espacio

In [ ]:
sudoku_board = get_random_puzzle()

print(np.matrix(sudoku_board))

result_backtracking_time = performance_check_time(sudoku_board,backtracking_sudoku_solver)
result_las_vegas_time = performance_check_time(sudoku_board,las_vegas_sudoku_solver)
result_mvr_time = performance_check_time(sudoku_board,csp_sudoku_solver)

[[4 7 8 9 6 2 0 0 0]
 [0 2 5 4 0 0 0 0 8]
 [6 9 0 0 3 5 7 0 0]
 [9 0 0 5 0 4 0 0 0]
 [0 0 4 0 7 0 5 0 9]
 [1 5 7 3 8 9 2 0 0]
 [8 0 0 1 0 0 0 7 5]
 [0 0 3 0 4 0 6 0 2]
 [0 4 9 0 0 6 0 8 3]]


In [ ]:
df = pd.DataFrame(
    {
        "Algoritmo":[
            "Backtracking",
            "Las Vegas",
            "CSP"
        ],
        "Tiempo (Segundos)":[
            result_backtracking_time,
            result_las_vegas_time,
            result_mvr_time
        ],
        "Cantidad de Nodos Expandidos":[
            expanded_nodes_bt,
            expanded_nodes_lv,
            expanded_nodes_csp,
        ],
    }
)
df

,Algoritmo,Tiempo (Segundos),Cantidad de Nodos Expandidos
0,Backtracking,0.001152,95
1,Las Vegas,0.000020,126
2,CSP,0.000019,55


Realizamos pruebas de rendimiento (Tiempo) para los algoritmos resolviendo 999 Tableros


---



In [ ]:
start_time = time.time()
for puzzle in puzzles:
  backtracking_sudoku_solver(extract_sudoku(puzzle))
end_time = time.time()

final_time_bt = end_time-start_time
print(f"Tiempo para {len(puzzles)} tableros {final_time_bt} segundos usando Backtracking")

Tiempo para 999 tableros 13.80496335029602 segundos usando Backtracking


In [ ]:
start_time = time.time()
for puzzle in puzzles:
  las_vegas_sudoku_solver(extract_sudoku(puzzle))
end_time = time.time()

final_time_lv = end_time-start_time
print(f"Tiempo para {len(puzzles)} tableros {final_time_lv} segundos usando Las Vegas")

Tiempo para 999 tableros 18.024020671844482 segundos usando Las Vegas


In [ ]:
start_time = time.time()
for puzzle in puzzles:
  csp_sudoku_solver(extract_sudoku(puzzle))
end_time = time.time()

final_time_csp = end_time-start_time
print(f"Tiempo para {len(puzzles)} tableros {final_time_csp} segundos usando Heuristica")

Tiempo para 999 tableros 6.023550271987915 segundos usando Heuristica


In [ ]:
df_purete = pd.DataFrame(
    {
        "Algoritmo":[
            "Backtracking",
            "Las Vegas",
            "CSP"
        ],
        "Tiempo (Segundos)":[
            final_time_bt,
            final_time_lv,
            final_time_csp,
        ],
        "Cantidad de Nodos Expandidos":[
            expanded_nodes_bt,
            expanded_nodes_lv,
            expanded_nodes_csp,
        ],
    }
)
df_purete

,Algoritmo,Tiempo (Segundos),Cantidad de Nodos Expandidos
0,Backtracking,13.804963,795861
1,Las Vegas,18.024021,810504
2,CSP,6.023550,46442


Se prueba con casos generales de N

In [ ]:
# def possible_values_general(board, row, col, N):
#     values = set(range(1, N*N + 1))
#     values -= set(board[row])
#     values -= set([board[i][col] for i in range(N*N)])
#     start_row, start_col = N * (row // N), N * (col // N)
#     values -= set([board[start_row + i][start_col + j] for i in range(N) for j in range(N)])

#     return values

# def most_constrained_cell_general(board, N):
#     min_possible_values = N*N + 1
#     mcv = None

#     for i in range(N*N):
#         for j in range(N*N):
#             if board[i][j] == 0:
#                 num_possible_values = len(possible_values_general(board, i, j, N))
#                 if num_possible_values < min_possible_values:
#                     min_possible_values = num_possible_values
#                     mcv = (i, j)

#     return mcv

# def csp_sudoku_solver_general(board, N):
#     mcv = most_constrained_cell_general(board, N)
#     if not mcv:
#         return True
#     row, col = mcv

#     for num in possible_values_general(board, row, col, N):
#         board[row][col] = num
#         if csp_sudoku_solver_general(board, N):
#             return True
#         board[row][col] = 0
#     return False

In [ ]:
# tablero = input("Cual es el tablero? (Ejemplo: 123456)")
# print()

# n = len(tablero) ** 0.5

# try:
#   if(int(n)<= 0):
#     print("El minimo debe ser 3")
#   if (int(n) % 3 != 0):
#     print("Debe ser un multiplo de 3!!")
# except:
#   print("Debe ser un numero")

# n_value = int(n)

# sudoku_board = extract_sudoku(tablero)

# print("Tablero inicial")
# print(np.matrix(sudoku_board))

# print_solution(backtracking_sudoku_solver,sudoku_board)
# print_solution(csp_sudoku_solver_general,sudoku_board)

